In [2]:
!pip install faiss-cpu sentence-transformers transformers pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 18.6 MB/s eta 0:00:00


In [3]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from pypdf import PdfReader
from google.colab import files

In [5]:
print("Please upload your PDF file...")
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
print(f"Uploaded: {pdf_path}")

# Extract text from PDF

reader = PdfReader(pdf_path)
pdf_text = ""
for page in reader.pages:
    if page.extract_text():
        pdf_text += page.extract_text() + "\n"

chunks = [chunk.strip() for chunk in pdf_text.split("\n") if chunk.strip()]

print(f"Extracted {len(chunks)} chunks from PDF")
print("Preview:\n", "\n".join(chunks[:5]))

Please upload your PDF file...


Saving ai_ml_notes.pdf to ai_ml_notes (1).pdf
Uploaded: ai_ml_notes (1).pdf
Extracted 12 chunks from PDF
Preview:
 Artificial Intelligence and Machine Learning Notes
Artificial Intelligence (AI) is the simulation of human intelligence in machines.
It allows systems to perform tasks that typically require human intelligence.
Examples: reasoning, problem-solving, learning, and language understanding.
Machine Learning


In [6]:
# Create embeddings + FAISS index
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print("Vector database created with", index.ntotal, "chunks")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector database created with 12 chunks


In [7]:
# Retrieval function
def retrieve(query, top_k=3):
    query_vec = embedder.encode([query])
    distances, indices = index.search(np.array(query_vec), top_k)
    retrieved_chunks = [chunks[i] for i in indices[0]]
    return retrieved_chunks

In [8]:
# Answer generation
generator = pipeline("text2text-generation", model="google/flan-t5-base")

def rag_pipeline(query):
    context = "\n".join(retrieve(query))
    prompt = f"Answer the question using the context below:\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    result = generator(prompt, max_length=200, do_sample=True)
    return result[0]["generated_text"]


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [9]:
# Ask Questions
while True:
    q = input("\nEnter your question (or type 'exit' to quit): ")
    if q.lower() == "exit":
        break
    print("\nRetrieved:", retrieve(q))
    print("Answer:", rag_pipeline(q))


Enter your question (or type 'exit' to quit): what is ai?

Retrieved: ['Artificial Intelligence (AI) is the simulation of human intelligence in machines.', 'Machine Learning (ML) is a subset of AI.', 'It allows systems to perform tasks that typically require human intelligence.']


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: simulation of human intelligence in machines

Enter your question (or type 'exit' to quit): what are types of machine learning

Retrieved: ['Types of ML include supervised learning, unsupervised learning, and reinforcement learning.', 'Machine Learning (ML) is a subset of AI.', 'Machine Learning']


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: supervised learning, unsupervised learning, and reinforcement learning

Enter your question (or type 'exit' to quit): exit
